In [1]:
import pandas as pd
import folium
from pyproj import Transformer

In [2]:
stations = pd.read_csv('netatmo_stations.csv', delimiter=',')
stations.head()

,device_id,module_id,lon,lat
0,70:ee:50:af:48:f0,02:00:00:af:94:7c,-6.160594,37.534757
1,70:ee:50:af:4f:b6,02:00:00:af:99:b2,-5.893018,37.743160
2,70:ee:50:65:03:ec,02:00:00:65:2c:5e,-6.090763,37.876842
3,70:ee:50:a4:53:92,02:00:00:9d:1a:c2,-5.924903,37.158357
4,70:ee:50:83:94:74,02:00:00:83:92:28,-5.769310,37.188882


In [3]:
zgz_coords = (41.653858, -0.876258)
map = folium.Map(location=zgz_coords, zoom_start=13)

# Iterate over each station
for index, station in stations.iterrows():
    lon = station['lon']
    lat = station['lat']

    # Add a marker for the point
    folium.Marker([lat, lon], popup=station['device_id']).add_to(map)

# If running in a Jupyter Notebook, display the map inline
map

In [4]:
'''stations = [
    '70:ee:50:7a:c2:72', # Camino de Sanlúcar
    #'70:ee:50:71:59:b0', # La Motilla
    '70:ee:50:73:e2:d8', # Dos Hermanas
    '70:ee:50:04:a4:52', # San José de la Rinconada 1
    '70:ee:50:7f:a6:e0', # San José de la Rinconada 2
    '70:ee:50:32:ae:72', # Coria del Río
    '70:ee:50:06:91:04', # La Puebla del Río
    '70:ee:50:65:37:e0', # Almensilla
]'''

"stations = [\n    '70:ee:50:7a:c2:72', # Camino de Sanlúcar\n    #'70:ee:50:71:59:b0', # La Motilla\n    '70:ee:50:73:e2:d8', # Dos Hermanas\n    '70:ee:50:04:a4:52', # San José de la Rinconada 1\n    '70:ee:50:7f:a6:e0', # San José de la Rinconada 2\n    '70:ee:50:32:ae:72', # Coria del Río\n    '70:ee:50:06:91:04', # La Puebla del Río\n    '70:ee:50:65:37:e0', # Almensilla\n]"

### Generate shapefile

In [5]:
import geopandas as gpd
from shapely.geometry import Point

In [6]:
# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(stations['lon'], stations['lat'])]
gdf = gpd.GeoDataFrame(stations, geometry=geometry)

# Set CRS and save
filename = 'netatmo_stations.shp'
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs('EPSG:25830')
gdf.to_file(filename)

### Locate stations

In [7]:
station_id = '70:ee:50:7a:c2:72'

row = stations[stations['device_id'] == station_id]

# Extract lon and lat
lon, lat = row[['lon', 'lat']].values[0]

location_map = folium.Map(location=zgz_coords, zoom_start=13)
folium.Marker([lat, lon], popup=station_id).add_to(location_map)
location_map